In [ ]:

!pip install yolov5


In [ ]:
# Include all packages
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import add, concatenate, BatchNormalization, LeakyReLU, ZeroPadding2D
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.metrics import MeanIoU
import torch.optim as optim

import pandas as pd
from yolov5.models.yolo import Model
# from yolov5.models.common import CSPDarknet53


In [ ]:
!git clone https: // github.com/ultralytics/yolov5.git


In [ ]:
!pip install - r yolov5/requirements.txt


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
#  import zipfile
# with zipfile.ZipFile('/content/drive/MyDrive/DL/FinalProject/signDatabasePublicFramesOnly.zip', 'r') as zip_ref:
#     zip_ref.extractall('./DataSet')


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:  # If you have at least one "configured" GPU, let's use it; otherwise, pass
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU")


In [ ]:
def LoadDataSet(dataSetFolderPath: str):
    images = []
    newAnnotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    width, height = 416, 416
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        # cv2.imshow("image", image)
        resizedImage = cv2.resize(
            image, (width, height), interpolation=cv2.INTER_AREA)
        # cv2.imshow("resized_img", resized_img)
        images.append(resizedImage)
        x1, y1, x2, y2 = row[2], row[3], row[4], row[5]
        x1, y1 = x1 / image.shape[1], y1 / image.shape[0]
        x2, y2 = x2 / image.shape[1], y2 / image.shape[0]
        newAnnotations.append([x1, y1, x2, y2])
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique()
    del annotationsDataFrame
    X_train, X_val, y_train, y_val = train_test_split(
        images, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val, uniqueSigns


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image


class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data  # data should be a list of (input, label) pairs
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_data, label = self.data[idx]

        if self.transform:
            input_data = self.transform(input_data)
        input_data = torch.from_numpy(input_data).float()

        return input_data, label


In [ ]:
def targets_to_tensors(targets, num_anchors, grid_sizes, num_classes):
    batch_size = int(targets[:, 0].max()) + 1
    target_obj = []
    target_cls = []
    target_box = []

    for grid_size in grid_sizes:
        target_obj.append(torch.zeros(
            (batch_size, num_anchors, grid_size, grid_size)))
        target_cls.append(torch.zeros(
            (batch_size, num_anchors, grid_size, grid_size, num_classes)))
        target_box.append(torch.zeros(
            (batch_size, num_anchors, grid_size, grid_size, 4)))

    for target in targets:
        batch_index, cls, x_center, y_center, width, height = target.long()

        for i, grid_size in enumerate(grid_sizes):
            x_cell, y_cell = int(
                x_center * grid_size), int(y_center * grid_size)
            anchor = 0  # Assuming only one anchor per grid cell for simplicity

            target_obj[i][batch_index, anchor, y_cell, x_cell] = 1
            target_cls[i][batch_index, anchor, y_cell, x_cell, cls] = 1
            target_box[i][batch_index, anchor, y_cell, x_cell] = torch.tensor(
                [x_center, y_center, width, height])

    return target_obj, target_cls, target_box


In [ ]:
import torch
import torch.nn as nn


class YOLOv5Loss(nn.Module):
    def __init__(self, num_classes, num_anchors=3):
        super(YOLOv5Loss, self).__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors

    def forward(self, preds, targets):
        obj_loss = torch.tensor(0.0, device=preds[0].device)
        cls_loss = torch.tensor(0.0, device=preds[0].device)
        box_loss = torch.tensor(0.0, device=preds[0].device)

        grid_sizes = [pred.size(2) for pred in preds]
        target_obj_list, target_cls_list, target_box_list = targets_to_tensors(
            targets, self.num_anchors, grid_sizes, self.num_classes)

        for i, pred in enumerate(preds):
            pred_obj = pred[..., 4].sigmoid()
            pred_cls = pred[..., 5:].sigmoid()
            pred_box = pred[..., :4]

            target_obj = target_obj_list[i].to(pred.device)
            target_cls = target_cls_list[i].to(pred.device)
            target_box = target_box_list[i].to(pred.device)

            obj_loss += nn.BCEWithLogitsLoss()(pred[..., 4], target_obj)
            cls_loss += nn.BCEWithLogitsLoss()(pred[..., 5:], target_cls)
            box_loss += nn.MSELoss()(pred_box, target_box)

        total_loss = obj_loss + cls_loss + box_loss
        return total_loss


In [ ]:
import torch
import torch.nn as nn


def setup_yolov5_loss_and_metric(num_classes, num_anchors):
    # Define the loss components
    objectness_loss = nn.BCELoss()
    class_loss = nn.BCELoss()
    bbox_loss = nn.MSELoss()
    mean_iou = MeanIoU(num_classes=num_classes)

    def yolov5_loss(y_true, y_pred):
        # Extract the ground truth and predictions for objectness, class, and bbox
        y_true_objectness, y_true_class, y_true_bbox = torch.split(
            y_true, [num_anchors, num_classes, 4], dim=-1)
        y_pred_objectness, y_pred_class, y_pred_bbox = torch.split(
            y_pred, [num_anchors, num_classes, 4], dim=-1)

        # Calculate the individual losses
        objectness_loss_value = objectness_loss(
            y_pred_objectness, y_true_objectness)
        class_loss_value = class_loss(y_pred_class, y_true_class)
        bbox_loss_value = bbox_loss(y_pred_bbox, y_true_bbox)

        # Combine the losses
        total_loss = objectness_loss_value + class_loss_value + bbox_loss_value
        return total_loss

    return yolov5_loss, mean_iou


In [ ]:
def _create_yolov5(input_shape, num_classes, version="s"):
    cfg = f"yolov5/models/yolov5{version}.yaml"
    model = Model(cfg, ch=3, nc=num_classes)
    return model


In [ ]:
X_train, X_val, y_train, y_val, uniqueSigns = LoadDataSet("./DataSet")


In [ ]:
trainDataSet = []  # List of (input, label) pairs for the training set
valDataSet = []  # List of (input, label) pairs for the validation set
for i in range(len(X_train)):
    trainDataSet.append((X_train[i], y_train[i]))

for i in range(len(X_val)):
    valDataSet.append((X_val[i], y_val[i]))
# Create the datasets
trainDataSetset = CustomDataset(trainDataSet)
valDataSetset = CustomDataset(valDataSet)

# Create the data loaders
trainDataSetloader = DataLoader(
    trainDataSetset, batch_size=32, shuffle=True, num_workers=4)
valDataSetloader = DataLoader(
    valDataSetset, batch_size=32, shuffle=False, num_workers=4)
input_shape = (416, 416, 3)
num_classes = len(uniqueSigns)
print(num_classes)


In [ ]:
yolov5_model = _create_yolov5(input_shape, num_classes)


In [ ]:
optimizer = optim.Adam(yolov5_model.parameters(), lr=0.001)


In [ ]:
num_anchors = 3
# yolov5_loss, mean_iou = setup_yolov5_loss_and_metric( num_classes, num_anchors)
yolov5_loss = YOLOv5Loss(num_classes=num_classes)

mean_iou = MeanIoU(num_classes=num_classes)


In [ ]:
import time

num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
yolov5_model = yolov5_model.to(device)

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print('-' * 10)

    start_time = time.time()

    for phase in ['train']:
        if phase == 'train':
            yolov5_model.train()  # Set model to training mode
            data_loader = trainDataSetloader
        else:
            yolov5_model.eval()  # Set model to evaluate mode
            data_loader = valDataSetloader
        data_loaderLength = len(data_loader)
        running_loss = 0.0
        running_metric = 0.0

        # Iterate over data.
        for inputs, labels in data_loader:
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = inputs.to(device)
            labels = [torch.tensor(label).to(device) for label in labels]

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            with torch.set_grad_enabled(phase == 'train'):
                outputs = yolov5_model(inputs)
                # print("Outputs shapes:")
                # for output in outputs:
                #     print(output.shape)
                # print("Labels shapes:")
                # for label in labels:
                #     print(label.shape)
                loss = yolov5_loss(outputs, labels)
                # metric_value = mean_iou(outputs, labels)

                # Backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            # running_metric += metric_value.item() * inputs.size(0)

        epoch_loss = running_loss / data_loaderLength
        # epoch_metric = running_metric / len(data_loader.dataset)

        print(f"{phase} Loss: {epoch_loss:.4f}")
        print(f"{phase} took {(time.time() - start_time):.1f} seconds")

    print()

print("Training complete.")
